In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, f1_score
import io
from helper import *

from sklearn.model_selection import GridSearchCV

c:\Users\scale\miniconda3\envs\{tp-redes}\Lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import mlflow
import mlflow.pytorch

In [5]:
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils

In [6]:
mlflow.set_experiment("Clasificador_Imagenes")

<Experiment: artifact_location='file:///c:/Users/scale/OneDrive/Desktop/skin-dataset-classification-main/mlruns/121373703533897100', creation_time=1764024673716, experiment_id='121373703533897100', last_update_time=1764024673716, lifecycle_stage='active', name='Clasificador_Imagenes', tags={}>

In [7]:
def log_classification_report(model, loader, writer, device, classes, step, prefix="val"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    fig_cm, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
    ax.set_title(f'{prefix.title()} - Confusion Matrix')

    # Guardar localmente y subir a MLflow
    fig_path = f"confusion_matrix_{prefix}_epoch_{step}.png"
    fig_cm.savefig(fig_path)
    mlflow.log_artifact(fig_path)
    os.remove(fig_path)

    plot_to_tensorboard(fig_cm, writer, f"{prefix}/confusion_matrix", step)

    cls_report = classification_report(all_labels, all_preds, target_names=classes)
    writer.add_text(f"{prefix}/classification_report", f"<pre>{cls_report}</pre>", step)

    # También loguear texto del reporte
    with open(f"classification_report_{prefix}_epoch_{step}.txt", "w") as f:
        f.write(cls_report)
    mlflow.log_artifact(f.name)
    os.remove(f.name)


In [8]:
# Entrenamiento y validación
def evaluate(model, loader, writer, device, classes, epoch=None, prefix="val"):
    log_classification_report(model, loader, writer, device, classes, step=epoch , prefix=prefix)
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss_sum += loss.item()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            if i == 0 and epoch is not None:
                img_grid = vutils.make_grid(images[:8].cpu(), normalize=True)
                writer.add_image(f"{prefix}/images", img_grid, global_step=epoch)

    acc = 100.0 * correct / total
    avg_loss = loss_sum / len(loader)
    f1 = f1_score(all_labels, all_preds, average='macro')  # o 'weighted' si tenés clases desbalanceadas

    if epoch is not None:
        writer.add_scalar(f"{prefix}/loss", avg_loss, epoch)
        writer.add_scalar(f"{prefix}/accuracy", acc, epoch)
        writer.add_scalar(f"{prefix}/f1_score", f1, epoch)

    return avg_loss, acc, f1


In [9]:
# Paths
train_dir = "data/Split_smol/train"
val_dir = "data/Split_smol/val/"

In [10]:
# Crear directorio de logs de TensorBoard con identificador único por modelo
log_dir = f"runs/experimentos_skin_clean"
writer = SummaryWriter(log_dir=log_dir)

In [11]:
np.random.rand()

0.9314940364764521

In [15]:
hparams_space= {
    "input_size":  [64, 128], 
    "model": ("CNNClassifier"),
    "batch_size": [16,64,128],
    "lr": [1e-2,1e-3,1e-4],
    "optimizer": ["Adam", "SGD"],      
    "HFlip": [0.5, 0.0],        
    "VFlip": [0.0],             
    "RBContrast": [0.5, 0.0], 
    "loss_fn": "CrossEntropyLoss",
    "dropout": [0.2, 0.3, 0.4],     
    "es_patience": 10,         
    "epochs": 100               
}

In [ ]:
stop_all = False
modelnbr = 0

for input_size in hparams_space["input_size"]:
    if stop_all: break
    for batch_size in hparams_space["batch_size"]:
        if stop_all: break
        for lr in hparams_space["lr"]:
            if stop_all: break
            for optimizer in hparams_space["optimizer"]:
                if stop_all: break
                for HFlip in hparams_space["HFlip"]:
                    if stop_all: break
                    for VFlip in hparams_space["VFlip"]:
                        if stop_all: break
                        for RBContrast in hparams_space["RBContrast"]:
                            if stop_all: break
                            for dropout in hparams_space["dropout"]:
                                if stop_all: break
                                if np.random.rand() < 0.02:
                                    print(f"modelo número: {modelnbr}", end = "\r")
                                    modelnbr += 1
                                    hparams= {
                                        "model": ("CNNClassifier"),
                                        "input_size":  input_size,
                                        "batch_size": batch_size,
                                        "lr": lr,
                                        "epochs": 200,
                                        "optimizer": optimizer,
                                        "HFlip": HFlip,
                                        "VFlip": VFlip,
                                        "RBContrast": RBContrast,
                                        "loss_fn": "CrossEntropyLoss",
                                        "train_dir": train_dir,
                                        "val_dir": val_dir,
                                        "es_patience": 5,
                                        "dropout": dropout,
                                    }
                                    train_transform = A.Compose([
                                    A.Resize(hparams["input_size"], hparams["input_size"]),
                                    A.HorizontalFlip(p=hparams["HFlip"]),
                                    # Remover VerticalFlip si no tiene sentido
                                    A.RandomBrightnessContrast(p=hparams["RBContrast"]),
                                    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
                                    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.5),
                                    A.Normalize(),
                                    ToTensorV2()
                                    ])
                                    val_test_transform = A.Compose([
                                        A.Resize(hparams["input_size"], hparams["input_size"]),
                                        A.Normalize(),
                                        ToTensorV2()
                                    ])
                                    train_dataset = CustomImageDataset(train_dir, transform=train_transform)
                                    val_dataset   = CustomImageDataset(val_dir, transform=val_test_transform)
                                    batch_size = hparams["batch_size"]
                                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                                    val_loader   = DataLoader(val_dataset, batch_size=batch_size)
                                    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                                    num_classes = len(set(train_dataset.labels))
                                    model = CNNClassifier(num_classes=num_classes, input_size = hparams["input_size"], dropout = hparams["dropout"]).to(device)
                                    criterion = nn.CrossEntropyLoss()
                                    optimizer = optim.Adam(model.parameters(), lr=hparams["lr"]) if hparams["optimizer"]=="Adam" else optim.SGD(model.parameters(), lr=hparams["lr"])
                                    scheduler = torch.optim.lr_scheduler.OneCycleLR(
                                        optimizer,
                                        max_lr=hparams["lr"],
                                        steps_per_epoch=len(train_loader),
                                        epochs=hparams["epochs"]
                                    )
                                    hparams["count_params"] = count_parameters(model)
                                    with mlflow.start_run():
                                        # Log hiperparámetros
                                        mlflow.log_params(hparams)
                                        best_val_acc = 0
                                        best_val_loss = 0
                                        best_train_acc = 0
                                        best_train_loss = 0
                                        best_epoch = 0
                                        for epoch in range(hparams["epochs"]):
                                            model.train()
                                            running_loss = 0.0
                                            correct, total = 0, 0
                                        
                                            for images, labels in train_loader:
                                                images, labels = images.to(device), labels.to(device)
                                        
                                                optimizer.zero_grad()
                                                outputs = model(images)
                                                loss = criterion(outputs, labels)
                                                loss.backward()
                                                optimizer.step()
                                                scheduler.step()
                                                running_loss += loss.item()
                                                _, preds = torch.max(outputs, 1)
                                                correct += (preds == labels).sum().item()
                                                total += labels.size(0)
                                        
                                            train_loss = running_loss / len(train_loader)
                                            train_acc = 100.0 * correct / total
                                            val_loss, val_acc, val_f1 = evaluate(model, val_loader, writer, device, train_dataset.label_encoder.classes_, epoch=epoch, prefix="val")

                                            mlflow.log_metrics({
                                                "train_loss": train_loss,
                                                "train_accuracy": train_acc,
                                                "val_loss": val_loss,
                                                "val_accuracy": val_acc,
                                                "val_f1_score": val_f1
                                            }, step=epoch)
                                                                                        

                                            print(f"Epoch {epoch+1}:")
                                            print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
                                            print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
                                        
                                            writer.add_scalar("train/loss", train_loss, epoch)
                                            writer.add_scalar("train/accuracy", train_acc, epoch)
                                            
                                            if val_acc > best_val_acc:
                                                best_val_acc = val_acc
                                                best_val_loss = val_loss
                                                best_train_acc = train_acc
                                                best_train_loss = train_loss
                                                best_epoch = epoch
                                                # Guardar modelo
                                                torch.save(model.state_dict(), "mlp_model.pth")
                                                print("Modelo guardado como 'mlp_model.pth'")
                                                mlflow.log_artifact("mlp_model.pth")
                                                mlflow.pytorch.log_model(model, artifact_path="pytorch_model")
                                                ##if val_acc >= 65.0:
                                                if val_acc >= 90.0:
                                                    print(f"\nAccuracy de validación de {val_acc:.2f}% alcanzado. Cortando búsqueda.")
                                                    stop_all = True
                                                    break
                                            elif epoch > best_epoch + hparams["es_patience"]:
                                                print("Early Stopping")
                                                break
                                                
                                                
                                        mlflow.log_metrics({
                                        "train_loss": best_train_loss,
                                        "train_accuracy": best_train_acc,
                                        "val_loss": best_val_loss,
                                        "val_accuracy": best_val_acc,
                                        "best_epoch": best_epoch,
                                        "val_f1_score": val_f1
                                    }, step=epoch+1)


Epoch 1:úmero: 0
  Train Loss: 2.0386, Accuracy: 26.58%
  Val   Loss: 1.9364, Accuracy: 36.11%


NameError: name 'patience_counter' is not defined

In [ ]:
# ---------- Inicio del grid-search corregido ----------
stop_all = False
modelnbr = 0

for input_size in hparams_space["input_size"]:
    if stop_all: break
    for batch_size in hparams_space["batch_size"]:
        if stop_all: break
        for lr in hparams_space["lr"]:
            if stop_all: break
            for optimizer in hparams_space["optimizer"]:
                if stop_all: break
                for HFlip in hparams_space["HFlip"]:
                    if stop_all: break
                    for VFlip in hparams_space["VFlip"]:
                        if stop_all: break
                        for RBContrast in hparams_space["RBContrast"]:
                            if stop_all: break
                            for dropout in hparams_space["dropout"]:
                                if stop_all: break

                                # muestreo aleatorio de combinaciones (tu criterio original)
                                if np.random.rand() < 0.02:
                                    print(f"modelo número: {modelnbr}", end="\r")
                                    modelnbr += 1

                                    # Hiperparámetros del experimento
                                    hparams = {
                                        "model": "CNNClassifier",
                                        "input_size":  input_size,
                                        "batch_size": batch_size,
                                        "lr": lr,
                                        "epochs": 200,
                                        "optimizer": optimizer,
                                        "HFlip": HFlip,
                                        "VFlip": VFlip,
                                        "RBContrast": RBContrast,
                                        "loss_fn": "CrossEntropyLoss",
                                        "train_dir": train_dir,
                                        "val_dir": val_dir,
                                        "es_patience": 5,
                                        "dropout": dropout,
                                    }

                                    # Transforms
                                    train_transform = A.Compose([
                                        A.Resize(hparams["input_size"], hparams["input_size"]),
                                        A.HorizontalFlip(p=hparams["HFlip"]),
                                        # A.VerticalFlip(p=hparams["VFlip"]),  # activá si tiene sentido
                                        A.RandomBrightnessContrast(p=hparams["RBContrast"]),
                                        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
                                        A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.5),
                                        A.Normalize(),
                                        ToTensorV2()
                                    ])
                                    val_test_transform = A.Compose([
                                        A.Resize(hparams["input_size"], hparams["input_size"]),
                                        A.Normalize(),
                                        ToTensorV2()
                                    ])

                                    # Datasets y loaders
                                    train_dataset = CustomImageDataset(train_dir, transform=train_transform)
                                    val_dataset   = CustomImageDataset(val_dir, transform=val_test_transform)
                                    batch_size = hparams["batch_size"]
                                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                                    val_loader   = DataLoader(val_dataset, batch_size=batch_size)

                                    # Device, modelo y optimizador
                                    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                                    num_classes = len(set(train_dataset.labels))
                                    model = CNNClassifier(num_classes=num_classes,
                                                          input_size=hparams["input_size"],
                                                          dropout=hparams["dropout"]).to(device)

                                    criterion = nn.CrossEntropyLoss()
                                    optimizer = (optim.Adam(model.parameters(), lr=hparams["lr"])
                                                 if hparams["optimizer"] == "Adam"
                                                 else optim.SGD(model.parameters(), lr=hparams["lr"]))

                                    # Scheduler (OneCycleLR) — steps_per_epoch debe > 0
                                    steps_per_epoch = max(1, len(train_loader))
                                    scheduler = torch.optim.lr_scheduler.OneCycleLR(
                                        optimizer,
                                        max_lr=hparams["lr"],
                                        steps_per_epoch=steps_per_epoch,
                                        epochs=hparams["epochs"]
                                    )

                                    hparams["count_params"] = count_parameters(model)

                                    # ---------- START run ----------
                                    with mlflow.start_run():
                                        mlflow.log_params(hparams)

                                        # Inicializaciones importantes (corregidas)
                                        best_val_loss = float("inf")   # <-- antes estaba 0, bug crítico
                                        best_val_acc = 0.0
                                        best_train_loss = None
                                        best_train_acc = None
                                        best_epoch = 0
                                        best_val_f1 = 0.0
                                        patience_counter = 0
                                        val_f1 = 0.0  # valor por defecto si nunca se calcula (seguridad)

                                        for epoch in range(hparams["epochs"]):
                                            model.train()
                                            running_loss = 0.0
                                            correct, total = 0, 0

                                            for images, labels in train_loader:
                                                images, labels = images.to(device), labels.to(device)

                                                optimizer.zero_grad()
                                                outputs = model(images)
                                                loss = criterion(outputs, labels)
                                                loss.backward()
                                                optimizer.step()
                                                # scheduler.step() debe llamarse una vez por batch con OneCycleLR
                                                try:
                                                    scheduler.step()
                                                except Exception:
                                                    pass

                                                running_loss += loss.item()
                                                _, preds = torch.max(outputs, 1)
                                                correct += (preds == labels).sum().item()
                                                total += labels.size(0)

                                            # métricas de train
                                            train_loss = running_loss / max(1, len(train_loader))
                                            train_acc = 100.0 * (correct / max(1, total))

                                            # evaluación en validación (tu función evaluate debe devolver val_loss,val_acc,val_f1)
                                            val_loss, val_acc, val_f1 = evaluate(
                                                model,
                                                val_loader,
                                                writer,
                                                device,
                                                train_dataset.label_encoder.classes_,
                                                epoch=epoch,
                                                prefix="val"
                                            )

                                            # log por época
                                            mlflow.log_metrics({
                                                "train_loss": train_loss,
                                                "train_accuracy": train_acc,
                                                "val_loss": val_loss,
                                                "val_accuracy": val_acc,
                                                "val_f1_score": val_f1
                                            }, step=epoch)

                                            print(f"Epoch {epoch+1}:")
                                            print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
                                            print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

                                            writer.add_scalar("train/loss", train_loss, epoch)
                                            writer.add_scalar("train/accuracy", train_acc, epoch)
                                            writer.add_scalar("val/loss", val_loss, epoch)
                                            writer.add_scalar("val/accuracy", val_acc, epoch)
                                            writer.add_scalar("val/f1", val_f1, epoch)

                                            # ===== Early stopping basado en val_loss (robusto) =====
                                            if val_loss < best_val_loss:
                                                best_val_loss = val_loss
                                                best_epoch = epoch
                                                best_val_acc = val_acc
                                                best_train_acc = train_acc
                                                best_train_loss = train_loss
                                                best_val_f1 = val_f1

                                                # Guardar modelo con nombre único por corrida
                                                model_filename = f"best_model_{modelnbr}.pth"
                                                torch.save(model.state_dict(), model_filename)
                                                print(f"Modelo guardado (mejora val_loss) -> {model_filename}")
                                                mlflow.log_artifact(model_filename)
                                                # mlflow.pytorch.log_model guarda todo el objeto (opcional)
                                                try:
                                                    mlflow.pytorch.log_model(model, artifact_path=f"pytorch_model_{modelnbr}")
                                                except Exception:
                                                    # logging no crítico: continuar si falla
                                                    pass

                                                patience_counter = 0
                                            else:
                                                patience_counter += 1

                                            # Early stopping por paciencia
                                            if patience_counter >= hparams["es_patience"]:
                                                print("Early stopping por falta de mejora en val_loss")
                                                break

                                            # Opcional: criterio global para detener todo el grid-search (por ejemplo objetivo de rendimiento)
                                            # if val_acc >= SOME_TARGET: 
                                            #     stop_all = True
                                            #     print("Objetivo global alcanzado — deteniendo búsqueda")
                                            #     break

                                        # Fin loop de epochs: loguear mejores métricas encontradas en esta corrida
                                        mlflow.log_metrics({
                                            "best_train_loss": best_train_loss if best_train_loss is not None else -1,
                                            "best_train_accuracy": best_train_acc if best_train_acc is not None else -1,
                                            "best_val_loss": best_val_loss,
                                            "best_val_accuracy": best_val_acc,
                                            "best_epoch": best_epoch,
                                            "best_val_f1": best_val_f1
                                        }, step=epoch + 1)

# ---------- Fin del grid-search ----------


Epoch 1:úmero: 0
  Train Loss: 2.0271, Accuracy: 25.57%
  Val   Loss: 1.9066, Accuracy: 36.11%
Modelo guardado (mejora val_loss) -> best_model_1.pth


2025/11/25 17:45:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epoch 2:
  Train Loss: 1.6650, Accuracy: 36.49%
  Val   Loss: 1.7223, Accuracy: 40.56%
Modelo guardado (mejora val_loss) -> best_model_1.pth


2025/11/25 17:46:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epoch 3:
  Train Loss: 1.4387, Accuracy: 43.82%
  Val   Loss: 1.5688, Accuracy: 42.78%
Modelo guardado (mejora val_loss) -> best_model_1.pth


...

Epoch 17:
  Train Loss: 0.9902, Accuracy: 63.22%
  Val   Loss: 1.1361, Accuracy: 55.00%
Modelo guardado (mejora val_loss) -> best_model_4.pth


2025/11/25 18:43:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epoch 18:
  Train Loss: 0.9748, Accuracy: 60.49%
  Val   Loss: 1.1499, Accuracy: 53.89%
Epoch 19:
  Train Loss: 0.9813, Accuracy: 62.36%
  Val   Loss: 1.1214, Accuracy: 57.22%
Modelo guardado (mejora val_loss) -> best_model_4.pth


2025/11/25 18:43:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epoch 20:
  Train Loss: 0.9868, Accuracy: 59.48%
  Val   Loss: 1.1219, Accuracy: 53.33%
Epoch 21:
  Train Loss: 0.9182, Accuracy: 63.22%
  Val   Loss: 1.0919, Accuracy: 54.44%
Modelo guardado (mejora val_loss) -> best_model_4.pth


In [ ]:
%reload_ext tensorboard





In [ ]:
%tensorboard --logdir runs/experimento_skin_clean --port 16006


Reusing TensorBoard on port 16006 (pid 13676), started 0:00:02 ago. (Use '!kill 13676' to kill it.)